In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install lxml

In [ ]:
import logging
import random
from urllib.parse import urlparse
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup as bs
import time
import pickle
from tqdm.notebook import tqdm
import pandas as pd
import re
import warnings
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager

warnings.filterwarnings(action='ignore')
from selenium.common.exceptions import TimeoutException, NoSuchElementException

### * 직접 설정 * 
* 키워드 리스트는 변수명 keywords 이여야함
* eliminate_cafe 리스트 반드시 같이 실행해야함


In [ ]:
main_keywords = ['로봇청소기 | 로청 | 청소']

##### 날짜 및 제외 카페 홈 주소 설정

In [ ]:
#제외 하고 싶은 카페홈 주소_필수실행
eliminate_cafe = ['https://cafe.naver.com/clubng',
                'https://cafe.naver.com/outletshop',
                'https://cafe.naver.com/gjtnwls123',
                'https://cafe.naver.com/movie567',
                'https://cafe.naver.com/komandos',
                'https://cafe.naver.com/comicity',
                'https://cafe.naver.com/bk1009',
                'https://cafe.naver.com/bestofmobile',
                'https://cafe.naver.com/illnesses',
                'https://cafe.naver.com/simsll',
                'https://cafe.naver.com/nexonsfw',
                'https://cafe.naver.com/ipod5',
                'https://cafe.naver.com/goldenpeach',
                'https://cafe.naver.com/itop5',
                'https://cafe.naver.com/peopledisc',
                'https://cafe.naver.com/soho',
                'https://cafe.naver.com/tictoc',
                'https://cafe.naver.com/s7942',
                'https://cafe.naver.com/simsll',
                'https://cafe.naver.com/godofhighschooljjang',
                'https://cafe.naver.com/joonggonara',
                'https://cafe.naver.com/hongdaeholic',
                'https://cafe.naver.com/poohstory',
                'https://cafe.naver.com/glycosarang',
                'https://cafe.naver.com/appleiphone']##추후에 크롤링 시도할 카페

pattern = '|'.join(eliminate_cafe)

## 키워드 설정
- 키워드 입력하실 때 너무 많은 양을 한꺼번에 넣지 마시고, 분할해서 진행해주세요!!
- 한 키워드당 수집하는데 약 48분 걸린다고 하니 이 점 유의해서 진행해주세요!!!

In [ ]:
####### 청소경험 키워드 

#포털 광고 글 필터링 위함
#eliminate_ads_keyword='-광고 -특가 -할인 -세일 -수수료 -이벤트 -핫딜 -입주 -이사'
eliminate_ads_keyword = '-할인 -핫딜 -매도 -임대 -계약 -특가 -가이드 -업체 -입주 -이사'

#청소 관련 단어

clean_key =['스캔', '경로', '라이다', '실시간'] #앞으로 해야함!

keywords=[]

for main in main_keywords:
    for clean in clean_key:
        keywords.append(main + '+%2B' + clean +' ' + eliminate_ads_keyword)

keywords

## 1. Naver Cafe 수집

#### 1.2.2 날짜미적용버전

In [ ]:
import os
# 로그 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Chrome 옵션 설정
options = Options()
options.add_argument("--disable-backgrounding-occluded-windows")
options.add_argument("--disable-notifications") 
options.add_argument('--disable-popup-blocking')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')


# WebDriver 실행 (ChromeDriver 버전이 Chrome 브라우저 버전과 일치하는지 확인)
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

In [ ]:
#새로운 데이터 프레임 만들기 
#기존 데이터프레임에 추가할때는 생성코드 실행하지않고 불러온다음 넣고자 하는 loc의 index 시작 넘버를 index 변수에 할당
dataframe = pd.DataFrame(columns=['date', 'keyword', 'title', 'contents', 'comments' ,'site', 'url']) #-데이터프레임 만들기
index = 0  #데이터 프레임 index (ex. dataframe.loc[index] = inputdata)

In [ ]:
for key in tqdm(keywords, desc='키워드'):
    logging.info("[{}] 키워드 검색 중".format(key))
    progress = tqdm(desc="수집된게시물")  # total 없이 생성
        
    ############ URL 불러오기 ############
    url = 'https://search.naver.com/search.naver?ssc=tab.cafe.all&sm=tab_jum&query={}'.format(key)

    driver.get(url)  #url 기져오기
    driver.implicitly_wait(15) #웹페이지 로딩될때까지 대기
    time.sleep(1)
    # AppleScript로 크롬 창 백그라운드로 전환
    
    breaker=False
    start_n, end_n = 1, 31
    while not breaker:
        #할당된 xpath없을시 빠르게 멈추기위한 코드_1
        if breaker == True:
            break
            
        check_soup = bs(driver.page_source, 'lxml')
        check_len = len(check_soup.select('div.title_area'))
        ###_1
        
        for i in range(start_n, end_n):
            xpath = '//*[@id="main_pack"]/section/div[1]/ul/li[{}]/div/div[2]/div[2]/a'.format(i)
    
            ###할당된 xpath없을시 빠르게 멈추기위한 코드_2
            if i == check_len+1:
                breaker = True
                break
            ###_2
            try:
                site = driver.find_element(By.XPATH, xpath)
                href_value = site.get_attribute("href")
                
                if not any(element in href_value for element in eliminate_cafe):
                    site.click()
           
                    driver.implicitly_wait(15) 
                    time.sleep(random.uniform(0.5, 1))
            
                    ###########수집하기###########
                    driver.switch_to.window(driver.window_handles[1])
                    
                    
                    #제목 및 전체 로딩_중긴에 soup 전체가 수집되지않는 경우 발생
                    for i in range(5):
                        try:
                            title=''
                            time.sleep(random.uniform(0.4, 1))
                            driver.switch_to.frame('cafe_main') #메인프레임전환
                            driver.implicitly_wait(15)
                            soup = bs(driver.page_source, 'lxml')
                            title = soup.select('h3.title_text')[0].text
                            break
                        except:
                            driver.refresh()
                           
                    if len(title) != 0:     
                        #본문
                        contents = [i.text for i in soup.select('div.se-component-content')]
                        if len(contents) == 0:
                            contents = [i.text for i in soup.select('div.ContentRenderer')]
                        contents = ' '.join(contents)
                        contents = contents.replace('\u200b', ' ')
                        contents = re.sub(r'\s+', ' ', contents).strip() #중복 공백 제거

                        
                        #날짜
                        date = soup.select('span.date')[0].text
                        
                        #댓글
                        comments = [i.text for i in soup.select('span.text_comment')]
                        
                        #url
                        url = driver.current_url
                        #데이터프레임에 추가
                        input_data = [date, keywords, title, contents, comments, '네이버포탈' , url]
                        dataframe.loc[index] = input_data
                        index+=1
                        progress.update(1)
                    else:
                        print('here')
                    
                    #탭닫기
                    while True:
                        time.sleep(0.2)
                        if len(driver.window_handles) == 2:
                            driver.close()
                            break
                    #탭전환    
                    while True:
                        time.sleep(0.2)
                        if len(driver.window_handles) == 1:
                            driver.switch_to.window(driver.window_handles[0])
                            time.sleep(0.2)
                            break
            except:
                pass
                
            try:   #카페글이 삭제된 게시물입니다.
                driver.switch_to.alert.dismiss()
                all_tabs = driver.window_handles
                for tab in all_tabs[1:]:
                    driver.switch_to.window(tab)
                    time.sleep(0.2)
                    driver.close()
                driver.switch_to.window(all_tabs[0])
            except:
                pass
            
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        
        start_n+=30
        end_n+=30

In [ ]:
dataframe['keyword'][0]

In [ ]:
dataframe.head()

## 2. 저장

In [ ]:
#URL 기준 필터링
dataframe = dataframe.drop_duplicates(subset='url',keep='first').reset_index(drop=True)

In [ ]:
print(dataframe.shape)
dataframe.tail()

In [ ]:
# CSV 파일로 저장
csv_filename = "naver_potal_카테고리명_키워드명.csv"
dataframe.to_csv(csv_filename, encoding="utf-8-sig")
logging.info(f"데이터가 '{csv_filename}' 파일로 저장되었습니다.")